In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()
walks_nb=182183
generated_expts=1921314
rob_Stuff=1932246
mixed=1932027
pdf="GL1932384"


In [2]:
from rspace_loader import RSpaceLoader
loader =RSpaceLoader(url=os.getenv("RSPACE_URL"), api_key=os.getenv("RSPACE_API_KEY"), global_id="GL1932384")
docs = loader.load()

{'id': 1932384, 'globalId': 'GL1932384', 'name': 'polo.pdf', 'caption': None, 'contentType': 'application/pdf', 'created': '2023-09-30T20:16:00.731Z', 'size': 3260672, 'version': 1, '_links': [{'link': 'https://community.researchspace.com/api/v1/files/1932384', 'rel': 'self'}, {'link': 'https://community.researchspace.com/api/v1/files/1932384/file', 'rel': 'enclosure'}]}


In [5]:
len(docs)

13

In [ ]:
for d in docs:
    print(len(d.page_content))
docs[1]

In [10]:

from langchain.text_splitter import RecursiveCharacterTextSplitter

splitter = RecursiveCharacterTextSplitter(chunk_size=3500, chunk_overlap=200)
split_docs=splitter.split_documents(docs)
len(split_docs)
for l in split_docs:
    print (len(l.page_content))

3776
1479
3440
3911
300
2572
2843
3974
1076
3018
3875
1531
2945
3971
1091
3060
3935
892
3523
3403
3985
3933
1821


In [11]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.llm import LLMChain
from langchain.prompts import PromptTemplate
chat_llm = ChatOpenAI(temperature=0.0)
prompt = """
    Write a concise summary of the input text. If the input text is about multiple subjects, 
    summarise each subject.

    Extract a maximum of 10 keywords and list these at the end in a section called 'Keywords'.
    Generate keywords from amongst the different subjects in the document.
    
    The text:
    
    {text}
    
    Your summary:
    
    Your keywords (maximum 10):
    """
promptT = PromptTemplate(input_variables=['text'],template=prompt)
llm_chain = LLMChain(llm=chat_llm, prompt=promptT)

In [8]:
from langchain.chains.summarize import load_summarize_chain
from langchain.callbacks import OpenAICallbackHandler
import textwrap
openai_cb = OpenAICallbackHandler()


In [12]:

chain = load_summarize_chain(llm=chat_llm,  chain_type="map_reduce", combine_prompt=promptT)
output_summary = chain.run(split_docs[0:10], callbacks=[openai_cb])
openai_cb

Tokens Used: 9229
	Prompt Tokens: 8180
	Completion Tokens: 1049
Successful Requests: 11
Total Cost (USD): $0.014368000000000002

In [14]:
print (output_summary)

This study focuses on the role of the Polo kinase gene in cytokinesis during spermatogenesis in Drosophila. The researchers found that defects in cytokinesis resulted in reduced numbers of cells and abnormal structures in spermatocytes. They also observed failures in the localization of certain proteins involved in cytokinesis. The study suggests that Polo-like kinases play a conserved role in regulating progression through M phase and the early events of cytokinesis.

Keywords: Polo kinase gene, cytokinesis, spermatogenesis, Drosophila, defects, proteins, M phase, cell division, localization, Polo-like kinases.



The above works very well, it does a good job at summarising. 3.5 turbo seems good, much faster than gpt4. Interestingly the very simple prompt works much better than 
TODO:
 - do this in streamlit
 - input field for a notebook or folder id. Support global Id or normal ID
 - Load docs and summarise. Note how many docs there are (max 10)
 - generate summary of each doc
 - generate summary of summaries of folder.

In [ ]:
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores.utils import filter_complex_metadata
from lan
db = Chroma.from_documents(docs, OpenAIEmbeddings())



In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

pdfLoader = PyPDFLoader('./data/inc.pdf')
splitter = RecursiveCharacterTextSplitter(chunk_size=4000, chunk_overlap=400)
docs = pdfLoader.load_and_split(text_splitter=splitter);

In [ ]:
chars = sum(len(x.page_content) for x in docs)
len(docs)

In [ ]:
with open('polosummary-gpt4output.txt', 'r') as input:
    lines = input.readlines()

In [ ]:
from langchain.docstore.base import Document
content  = [''.join(lines)]
summarisedOnce = [Document(page_content=s1, metadata={'source':f"page-{i}"}) for i, s1 in enumerate(content)]
summarisedAndSplit= splitter.split_documents(summarisedOnce)
len(content[0])

In [ ]:
from langchain.chains.llm import LLMChain
outputs2 = []
for d in docs:
    resp = llm_chain.predict(text=d.page_content, research_area='biomedical')
    outputs2.append(resp)
    

In [ ]:
t = ''.join(outputs2)
d2 = Document(page_content=t, metadata=)

In [ ]:
d2_split =splitter.split_documents([d2])

In [ ]:
d2_split

In [ ]:
from langchain.chains.llm import LLMChain
outputs23 = []
for d in d2_split:
    resp = llm_chain.predict(text=d.page_content, research_area='biomedical')
    outputs23.append(resp)

In [ ]:
from langchain.chains.summarize import load_summarize_chain
import textwrap
chain = load_summarize_chain(llm=chat_llm, verbose=True, chain_type="map_reduce")
output_summary = chain.run(docs)
wrapped_text = textwrap.fill(output_summary, width=100)
print(wrapped_text)